<a href="https://colab.research.google.com/github/L-ENA/CORD19scripts/blob/master/CORD_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##squad file for evaluating immunoglobulin  and CORD-19 data
#imports and installs

import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

!pip install git+https://github.com/L-ENA/transformers

!pip install tensorboardX
!git clone https://github.com/L-ENA/transformers.git

!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
!pip install 
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Found GPU at: /device:GPU:0
  Cloning https://github.com/L-ENA/transformers to /tmp/pip-req-build-4vapbaw_
  Running command git clone -q https://github.com/L-ENA/transformers /tmp/pip-req-build-4vapbaw_
     |████████████████████████████████| 1.0MB 7.1MB/s 
     |████████████████████████████████| 870kB 42.0MB/s 
  Created wheel for transformers: filename=transformers-2.2.1-cp36-none-any.whl size=382564 sha256=6428244520386e03d5f3444f001b6be577c794f15038537ac173f385fe4c59db
  Stored in directory: /tmp/pip-ephem-wheel-cache-tup48hnl/wheels/88/72/14/9ac450fa11a8da19292c8649e29d7b544e1d52f223f7330f5e
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=c4ff124f59e873e352df1f96a9d31140151990cbd375eb87349eac509fe78dbd
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built transformers sacremoses
     |████████████████████████████████| 204kB 7.1MB/s 
Cloning into 'transformers'..

In [0]:
#To connect the google drive, where the training data is saved
#The fine-tuned model will also be saved there

from google.colab import drive#connect the gdrive with all data
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
###
#Script to run training on combined SQUAD and ebmnlp data
#
#model_name_or_path here we use bert large, this can be changed
#data_dir is the path where the training and testing data sit; for convenience they should be named dev-v2.0.json and train-v2.0.json
#version_2_with_negative means that we have the option to train the model to decide when to not give an answer, we can do that becasue the training data has some impossible answers marked up
#
#output_dir where the final model will be saved
#do_train tell the model that it should train its weights
###


!python /content/transformers/examples/run_squad.py \
  --model_type bert \
  --model_name_or_path bert-large-uncased \
  --do_eval \
  --version_2_with_negative \
  --data_dir /content/drive/My\ Drive/squadData/patients \
  --per_gpu_train_batch_size 18 \
  --learning_rate 3e-5 \
  --num_train_epochs 1.0 \
  --max_seq_length 150 \
  --doc_stride 128 \
  --logging_steps 1000 \
  --save_steps 1000 \
  --output_dir /content/drive/My\ Drive/SQUADS/sampled_large \
  --overwrite_output_dir \
  --do_train \
  --do_lower_case \


   
  

01/29/2020 08:54:17 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
01/29/2020 08:54:17 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.4c88e2dec8f8b017f319f6db2b157fee632c0860d9422e4851bd0d6999f9ce38
01/29/2020 08:54:17 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "o

In [0]:
####Eval Script for mining on cord
#
#This script uses my previously fin-tuned Question answering model (stored in /content/drive/My\ Drive/SQUADS/sampled_large/ ) to mine Conditions described in the Covid dataset.
#data_dir this folder needs to contain the CORD-data, as given by the conversion script in covid_input.py

!python /content/transformers/examples/run_squad.py \
  --model_type bert \
  --model_name_or_path /content/drive/My\ Drive/SQUADS/sampled_large/ \
  --do_eval \
  --version_2_with_negative \
  --do_lower_case \
  --data_dir /content/drive/My\ Drive/squadData/CORD19 \
  --per_gpu_train_batch_size 18 \
  --per_gpu_eval_batch_size 10 \
  --learning_rate 3e-5 \
  --num_train_epochs 1.0 \
  --max_seq_length 150 \
  --doc_stride 128 \
  --logging_steps 1000 \
  --save_steps 1000 \
  --output_dir /content/drive/My\ Drive/SQUADS/sampled_large \

 

04/01/2020 14:57:18 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/01/2020 14:57:18 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/SQUADS/sampled_large/config.json
04/01/2020 14:57:18 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

04/01/2020 14:57:18 - INFO - transformers.tokenization_utils - 

In [0]:
####Eval Script for Condition mining
!python /content/transformers/examples/run_squad.py \
  --model_type bert \
  --model_name_or_path /content/drive/My\ Drive/SQUADS/sampled_Conditions/ \
  --do_eval \
  --version_2_with_negative \
  --do_lower_case \
  --data_dir /content/drive/My\ Drive/squadData/CORD19 \
  --per_gpu_train_batch_size 18 \
  --per_gpu_eval_batch_size 10 \
  --learning_rate 3e-5 \
  --num_train_epochs 1.0 \
  --max_seq_length 150 \
  --doc_stride 128 \
  --logging_steps 1000 \
  --save_steps 1000 \
  --output_dir /content/drive/My\ Drive/SQUADS/sampled_Conditions \

#done 4 c

04/01/2020 15:31:56 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/01/2020 15:31:56 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/SQUADS/sampled_Conditions/config.json
04/01/2020 15:31:56 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

04/01/2020 15:31:56 - INFO - transformers.tokenization_uti

In [0]:
####Eval Script for Intervention mining
!python /content/transformers/examples/run_squad.py \
  --model_type bert \
  --model_name_or_path /content/drive/My\ Drive/SQUADS/200\ I/squadCheckpoints \
  --do_eval \
  --version_2_with_negative \
  --do_lower_case \
  --data_dir /content/drive/My\ Drive/squadData/CORD19/I \
  --per_gpu_train_batch_size 18 \
  --per_gpu_eval_batch_size 10 \
  --learning_rate 3e-5 \
  --num_train_epochs 1.0 \
  --max_seq_length 150 \
  --doc_stride 128 \
  --logging_steps 1000 \
  --save_steps 1000 \
  --output_dir /content/drive/My\ Drive/SQUADS/200\ I/squadCheckpoints \

#done 4 c

04/01/2020 16:53:46 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/01/2020 16:53:46 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/SQUADS/200 I/squadCheckpoints/config.json
04/01/2020 16:53:47 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

04/01/2020 16:53:47 - INFO - transformers.tokenization_